In [1]:
import pandas as pd 
import numpy as np 

In [14]:
train_data =pd.read_csv("PlayTennis.csv") 
train_data

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [9]:
#Calculating Entropy for each feature of the dataset
def feature_entropy(feature_value_data, target, class_list):
    entropy = 0 
    total_class_count = feature_value_data.shape[0] 
    for x in class_list:
        each_class_count = feature_value_data[feature_value_data[target] == x].shape[0] 
        class_entropy = 0
        if each_class_count != 0:
            class_p = each_class_count/total_class_count 
            class_entropy = - class_p * np.log2(class_p) 
        entropy += class_entropy
    return entropy

In [6]:
def info_gain(feature_name, train_data, target, class_list):
    feature_info_gain = 0
    feature_value_list = train_data[feature_name].unique()
    total_row = train_data.shape[0]
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value]
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = feature_entropy(feature_value_data, target, class_list)
        feature_value_p = feature_value_count/total_row
        feature_info_gain += feature_value_p * feature_value_entropy 
    return dataset_entropy(train_data, target, class_list) - feature_info_gain 

In [12]:
#Calculating Total_Entropy for each classes
def dataset_entropy(train_data, target, class_list):
    total_entropy = 0
    total_row = train_data.shape[0]    
    for x in class_list: 
        each_class_count = train_data[train_data[target] == x].shape[0] 
        each_class_entropy = - (each_class_count/total_row)*np.log2(each_class_count/total_row) 
        total_entropy += each_class_entropy
    return total_entropy
    

In [7]:
def best_feature(train_data, target, class_list):
    max_info_gain = 0
    max_info_feature = None 
    feature_list = train_data.columns.drop(target)                                     
    for feature in feature_list:
        feature_info_gain = info_gain(feature, train_data, target, class_list)
        if max_info_gain < feature_info_gain: 
            max_info_gain = feature_info_gain
            max_info_feature = feature          
    return max_info_feature

In [13]:
best_feature(train_data,'Play Tennis',train_data['Play Tennis'].unique())

'Outlook'

In [21]:
import pandas as pd 
import numpy as np 

#Importing Train_Dataset 
train_data =pd.read_csv("PlayTennis.csv") 
#print(train_data.head()) 

#Calculating Information_Gain
def info_gain(feature_name, train_data, target, class_list):
    feature_info_gain = 0
    feature_value_list = train_data[feature_name].unique()
    total_row = train_data.shape[0]
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value]
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = feature_entropy(feature_value_data, target, class_list)
        feature_value_p = feature_value_count/total_row
        feature_info_gain += feature_value_p * feature_value_entropy 
    return dataset_entropy(train_data, target, class_list) - feature_info_gain 

#Calculating Total_Entropy for each classes
def dataset_entropy(train_data, target, class_list):
    total_entropy = 0
    total_row = train_data.shape[0]    
    for x in class_list: 
        each_class_count = train_data[train_data[target] == x].shape[0] 
        each_class_entropy = - (each_class_count/total_row)*np.log2(each_class_count/total_row) 
        total_entropy += each_class_entropy
    return total_entropy
    
#Calculating Entropy for each feature of the dataset
def feature_entropy(feature_value_data, target, class_list):
    entropy = 0 
    total_class_count = feature_value_data.shape[0] 
    for x in class_list:
        each_class_count = feature_value_data[feature_value_data[target] == x].shape[0] 
        class_entropy = 0
        if each_class_count != 0:
            class_p = each_class_count/total_class_count 
            class_entropy = - class_p * np.log2(class_p) 
        entropy += class_entropy
    return entropy

#Finding the best feature that has the highest Information_Gain
def best_feature(train_data, target, class_list):
    max_info_gain = 0
    max_info_feature = None 
    feature_list = train_data.columns.drop(target)                                     
    for feature in feature_list:
        feature_info_gain = info_gain(feature, train_data, target, class_list)
        if max_info_gain < feature_info_gain: 
            max_info_gain = feature_info_gain
            max_info_feature = feature          
    return max_info_feature

#Building Branches of Decision_Tree                 
def sub_tree(feature_name, train_data, target, class_list):
    tree = {} 
    feature_value_count = train_data[feature_name].value_counts(sort=False)
    for feature_value, count in feature_value_count.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value]
        leaf_node = False 
        for x in class_list: 
            each_class_count = feature_value_data[feature_value_data[target] == x].shape[0] 
            if each_class_count == count: #Reaching to a leaf_node
                tree[feature_value] = x 
                train_data = train_data[train_data[feature_name] != feature_value] 
                leaf_node = True
        if not leaf_node: 
            tree[feature_value] = "Branch" #Tree should be expanded
    return tree, train_data

#Building Decision_Tree
def ID3(root, parent_node, train_data, target, class_list):
    if train_data.shape[0] != 0: 
        max_info_feature = best_feature(train_data, target, class_list) 
        tree, train_data = sub_tree(max_info_feature, train_data, target, class_list) 
        next_root = None 
        if parent_node != None: 
            root[parent_node] = dict()
            root[parent_node][max_info_feature] = tree
            next_root = root[parent_node][max_info_feature]
        else: #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]      
        for node, branch in list(next_root.items()): 
            if branch == "Branch": #if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == node] 
                ID3(next_root, node, feature_value_data, target, class_list)

#Predicting New_new_instances for Test_dataset
def predict(tree, new_instance):
    if not isinstance(tree, dict): #Whether reaching a leaf_node or not
        return tree 
    else:
        root_node = next(iter(tree))
        feature_value = new_instance[root_node] 
        if feature_value in tree[root_node]: 
            return predict(tree[root_node][feature_value], new_instance) 
        else:
            return None
        
#Starting Training the dataset with making a Decicion_Tree 
target="Play Tennis"
tree = {} 
class_list = train_data[target].unique() 
ID3(tree, None, train_data, target, class_list)
print ('The final Decision_Tree is:',tree)

#Importing Test_Dataset for prediction of new_new_instances
test_data = pd.read_csv("PlayTennis_test.csv")

#Starting Testing new_data as Test_dataset & calculating Accuracy 
correct_count = 0
wrong_count = 0
for index, row in test_data.iterrows(): 
    result = predict(tree, test_data.iloc[index]) 
    if result == test_data[target].iloc[index]: 
        correct_count += 1 
    else:
        wrong_count+= 1 
accuracy =  correct_count / ( correct_count + wrong_count) 
print ('Accuracy=', accuracy)
                


The final Decision_Tree is: {'Outlook': {'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}, 'Overcast': 'Yes', 'Sunny': {'Humidity': {'Normal': 'Yes', 'High': 'No'}}}}
Accuracy= 1.0
